<a href="https://colab.research.google.com/github/theharshithh/onfinace-assignment/blob/main/openai_mod_embeds5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
!pip -q install langchain openai tiktoken chromadb langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain_cohere

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain_cohere import CohereEmbeddings


def read_doc(dir):
    file_loader = PyPDFDirectoryLoader(dir)
    docs = file_loader.load()
    return docs

loading data to /dataset dir

In [6]:
!pip install pypdf

In [ ]:
primary_data = read_doc('dataset/')
primary_data

creating chunks for the `primary_data`

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
spilit_texts = text_splitter.split_documents(primary_data)

## create the embeddings using hf ede by baai

In [11]:
# #hugging face bge embeddings
# model_name = "BAAI/bge-base-en"
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     encode_kwargs=encode_kwargs
# )

change of embeddings from `OpenAIEmbeddings()` to bge-embeddings

In [12]:
persist_directory = 'db-openai_modified'
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
#openai embeddings -> converts texts to vecs
embedding = OpenAIEmbeddings(openai_api_key = openai_api_key, model="text-embedding-3-small")

#load the spilt chunk vectors into the vectordb
vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [13]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [14]:

user_query = 'Considering 2 wheeler demand, who seems to be the segment winner?'

In [15]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents(user_query)
#getting the relevant docs for the user query
docs

[Document(page_content='5% YoY , largely due to  the impact of  unseasonal rains and low yield s for key \ncrops  such as oni on. In Mar’23, overall wholesale volumes for \nPV/CV/tractor/3Ws are estimated to grow by 1%/ 1.5%/1 %/8% YoY, while 2Ws \nare likely to decline 4% YoY.  \n\uf06e 2Ws:  Retails are expected to grow 4-6% YoY in Mar ’23 since urban demand is \nsteady and inquiries have improved in certain states during the ongoing Navratri \nseason . However, overall sales  are still do wn by 15 -17% v /s the pre-Covid  level \nas rural demand remain s weak.  Enquiry levels during the key festival s (Gudi \nPadwa and Ugadi) varied greatly among  states.  We have noted 2-6% YoY growth \nin enquiry levels in states like Gujarat, Delhi and Karnataka , while  the same \ndeclined by 4-8% in other key states like UP, Bihar  and Maharashtra . The i nitial \nresponse to HMCL’ s new launch Xoom (110 cc scooter) has been positi ve as it has \naccounted for 25-30% of the overall enquiries in

In [16]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

### LLM chain

In [17]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-4-turbo", temperature=1.2 ), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [18]:
def process_llm_response(llm_response):
    print("Decision:")
    print(llm_response['result'])

    print("\n Source Documents:")
    for source in llm_response["source_documents"]:
        print("- " + source.metadata['source'])

    print("\nRelevant Data:")
    for source in llm_response["source_documents"]:
        print(source.page_content)

In [19]:
behaviour = '''Think like an expert finance consultant from a top tier firm. Do all the research and analysis for this document.'''
prompt_template = 'Keep the answer crisp. Give me as much numbers as you can. Every Statement of yours should be backed with data'
stop_prompt = f'Dont hallucinate, if u dont know the answer, tell you dont know'
query = f"Here is the user query : {user_query}. Remember this: {behaviour}. Give me the answers like this: {prompt_template}. Keep this in mind - {stop_prompt}"
llm_response = qa_chain(query)
# llm_response
process_llm_response(llm_response)

Decision:
Based on the detailed assessment of the current trends in the 2-wheeler (2W) sector from the data provided, Hero MotoCorp emerges as a noticeable segment winner in terms of recent performance:

- **Retail Growth**: Hero MotoCorp has shown a notable improvement in enquiries during Mar ’23 driven by a strong response to the new HMCL Xoom model, which accounted for 25-30% of overall enquiries in the same period.
- **YoY Growth in Enquiries**: In regions like Gujarat, Delhi, and Karnataka, Hero witnessed a YoY enquiry growth of 2-6%, which aligns with the urban demand stability mentioned. 
- **Inventory Level**: Hero MotoCorp maintains a higher inventory level (50-55 days) relative to its competitors, which might indicate higher readiness to meet potential demand surges post-festive season.
- **Strong Financials Performance** (From year-end metrics relocation):
  - Total Units Sold: Hero MotoCorp sold a impressive volume of 434,165 units in the current period (YoY change of -3.6%